<a href="https://colab.research.google.com/github/sinemyvs/1dcnnlstm-EEG-DREAMER/blob/main/1d_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install TensorFlow

In [ ]:
# ----- Gerekli Kütüphanelerin Yüklenmesi (Colab'da genellikle gereksiz ama emin olmak için) -----
!pip install numpy scipy scikit-learn tensorflow

# Gerekli kütüphanelerin içe aktarılması
import numpy as np
import scipy.io
from scipy.signal import butter, filtfilt, resample
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Dropout, LSTM, Dense, Input
from tensorflow.keras.optimizers import RMSprop # Tensorflow'un yeni sürümlerinde tf.keras.optimizers.legacy.RMSprop olabilir
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
import os
import random

# ----- Google Colab için Drive Bağlantısı -----
from google.colab import drive
drive.mount('/content/drive', force_remount=True) # Yeniden bağlanmayı zorla

# ----- Yapılandırma Değişkenleri -----
DREAMER_MAT_PATH_DRIVE = '/content/drive/MyDrive/DREAMER.mat'

EEG_FS = 128
ECG_FS_ORIGINAL = 256
TARGET_FS = 128
LOWCUT = 4.0
HIGHCUT = 45.0
FILTER_ORDER = 5
MIN_SIGNAL_LENGTH_FOR_FILTER = 40

LABEL_THRESHOLD = 3
N_PARTICIPANTS = 23
N_TRIALS_PER_PARTICIPANT_EXPECTED = 18
TRAIN_TRIALS_MIN_REQUIRED = 5
TEST_TRIALS_MIN_REQUIRED = 2

LR = 0.0001
EPOCHS = 2 # Hızlı test için 2 yapabilirsiniz
BATCH_SIZE = 32

SEED = 42
os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

# ----- Yardımcı Fonksiyonlar -----

def butter_bandpass_filter(data, lowcut, highcut, fs, order=5, min_len_for_filter=MIN_SIGNAL_LENGTH_FOR_FILTER):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    if high >= 1.0: high = 0.999
    if low <= 0.0: low = 0.001
    if low >= high: return data
    if not isinstance(data, np.ndarray) or data.ndim < 1 or data.shape[0] <= min_len_for_filter : return data # Eklendi: data geçerli mi kontrolü
    try:
        b, a = butter(order, [low, high], btype='band')
        y = filtfilt(b, a, data, axis=0)
    except ValueError: return data
    return y

def load_and_preprocess_dreamer(mat_file_path):
    print(f"{mat_file_path} yükleniyor...")
    try:
        mat = scipy.io.loadmat(mat_file_path)
    except FileNotFoundError:
        print(f"HATA: {mat_file_path} bulunamadı!")
        return None, None, None, 0
    except Exception as e:
        print(f"HATA: {mat_file_path} yüklenirken bir hata oluştu: {e}")
        return None, None, None, 0

    dreamer_data_struct = mat['DREAMER'][0, 0]
    all_participants_data_list = []
    all_participants_valence_labels_list = []
    all_participants_arousal_labels_list = []
    max_len_overall = 0
    total_trials_processed_successfully = 0
    total_trials_skipped_short_signal = 0
    total_trials_skipped_access_error = 0

    for p_idx in range(N_PARTICIPANTS):
        print(f"Katılımcı {p_idx + 1}/{N_PARTICIPANTS} işleniyor...")

        if p_idx == 0: # Sadece ilk katılımcı için detaylı inceleme
            print(f"\n--- Katılımcı {p_idx+1} Veri Yapısı İncelemesi (Düzeltme Sonrası) ---")
            try:
                participant_struct_debug = dreamer_data_struct['Data'][0, p_idx]
                # 1. Katman
                eeg_stim_container_L1_debug = participant_struct_debug['EEG'][0,0]['stimuli']
                print(f"EEG L1 container tipi: {type(eeg_stim_container_L1_debug)}, şekli: {eeg_stim_container_L1_debug.shape if isinstance(eeg_stim_container_L1_debug, np.ndarray) else 'N/A'}")

                if isinstance(eeg_stim_container_L1_debug, np.ndarray) and eeg_stim_container_L1_debug.shape == (1,1):
                    # 2. Katman
                    eeg_stim_container_L2_debug = eeg_stim_container_L1_debug[0,0]
                    print(f"EEG L2 container (iç hücre dizisi) tipi: {type(eeg_stim_container_L2_debug)}, şekli: {eeg_stim_container_L2_debug.shape if isinstance(eeg_stim_container_L2_debug, np.ndarray) else 'N/A'}")

                    if isinstance(eeg_stim_container_L2_debug, np.ndarray) and eeg_stim_container_L2_debug.shape == (N_TRIALS_PER_PARTICIPANT_EXPECTED, 1):
                        # 3. Katman (asıl sinyal verisi)
                        eeg_actual_signal_cell_debug = eeg_stim_container_L2_debug[0,0] # İlk denemenin hücresi
                        print(f"EEG L2'nin ilk hücresinin (gerçek sinyal hücresi) tipi: {type(eeg_actual_signal_cell_debug)}")
                        if isinstance(eeg_actual_signal_cell_debug, np.ndarray):
                             print(f"EEG gerçek sinyal (ilk deneme) tipi: {type(eeg_actual_signal_cell_debug)}, şekli: {eeg_actual_signal_cell_debug.shape}")
                        else: # Belki .item() gerekir
                            try:
                                eeg_actual_signal_via_item = eeg_actual_signal_cell_debug.item()
                                print(f"EEG gerçek sinyal (ilk deneme, .item() ile) tipi: {type(eeg_actual_signal_via_item)}, şekli: {eeg_actual_signal_via_item.shape if isinstance(eeg_actual_signal_via_item, np.ndarray) else 'N/A'}")
                            except Exception as e_item_inner:
                                print(f"Gerçek sinyal hücresine .item() hatası: {e_item_inner}")
                print("--- Bitti: Veri Yapısı İncelemesi (Düzeltme Sonrası) ---\n")
            except Exception as e_debug_print:
                print(f"Debug yazdırma sırasında hata: {e_debug_print}")

        try:
            participant_struct = dreamer_data_struct['Data'][0, p_idx]
            valence_scores_for_participant = participant_struct['ScoreValence'][0,0].flatten()
            arousal_scores_for_participant = participant_struct['ScoreArousal'][0,0].flatten()

            # Düzeltilmiş Erişim:
            eeg_inner_cell_array = participant_struct['EEG'][0,0]['stimuli'][0,0]
            ecg_inner_cell_array = participant_struct['ECG'][0,0]['stimuli'][0,0]

        except (IndexError, KeyError, TypeError) as e_access_main: # TypeError eklendi (NoneType üzerinde işlem yapmaya çalışırsa)
            print(f"HATA: Katılımcı {p_idx+1} için ana veri alanlarına erişirken hata ({e_access_main}). Bu katılımcı atlanıyor.")
            all_participants_data_list.append([])
            all_participants_valence_labels_list.append(np.array([]))
            all_participants_arousal_labels_list.append(np.array([]))
            total_trials_skipped_access_error += N_TRIALS_PER_PARTICIPANT_EXPECTED
            continue

        participant_trials_data_list = []
        participant_trials_valence_labels = []
        participant_trials_arousal_labels = []

        if not (isinstance(eeg_inner_cell_array, np.ndarray) and \
                eeg_inner_cell_array.ndim == 2 and eeg_inner_cell_array.shape[1] == 1 and \
                eeg_inner_cell_array.shape[0] >= 1 and \
                isinstance(ecg_inner_cell_array, np.ndarray) and \
                ecg_inner_cell_array.ndim == 2 and ecg_inner_cell_array.shape[1] == 1 and \
                ecg_inner_cell_array.shape[0] == eeg_inner_cell_array.shape[0]):
            print(f"Katılımcı {p_idx+1} için iç stimuli hücre dizisi beklenmedik şekle/tipe sahip veya EEG/EKG uyuşmuyor. EEG: {eeg_inner_cell_array.shape if isinstance(eeg_inner_cell_array, np.ndarray) else type(eeg_inner_cell_array)}, ECG: {ecg_inner_cell_array.shape if isinstance(ecg_inner_cell_array, np.ndarray) else type(ecg_inner_cell_array)}. Atlanıyor.")
            all_participants_data_list.append([])
            all_participants_valence_labels_list.append(np.array([]))
            all_participants_arousal_labels_list.append(np.array([]))
            total_trials_skipped_access_error += eeg_inner_cell_array.shape[0] if isinstance(eeg_inner_cell_array, np.ndarray) else N_TRIALS_PER_PARTICIPANT_EXPECTED
            continue

        n_actual_trials = eeg_inner_cell_array.shape[0]

        for t_idx in range(n_actual_trials):
            eeg_stimuli_raw = None
            ecg_stimuli_raw = None
            try:
                eeg_cell_content = eeg_inner_cell_array[t_idx, 0]
                ecg_cell_content = ecg_inner_cell_array[t_idx, 0]

                # Hücre içeriği genellikle doğrudan numpy array'dir bu seviyede.
                if isinstance(eeg_cell_content, np.ndarray) and eeg_cell_content.ndim >= 2:
                    eeg_stimuli_raw = eeg_cell_content
                else: # .item() çok nadir gerekir, genellikle hata verir
                    # print(f"Katılımcı {p_idx+1}, Deneme {t_idx+1} EEG hücre içeriği beklenmedik tip: {type(eeg_cell_content)}")
                    total_trials_skipped_access_error += 1
                    continue # Bu denemeyi atla

                if isinstance(ecg_cell_content, np.ndarray) and ecg_cell_content.ndim >= 2:
                    ecg_stimuli_raw = ecg_cell_content
                else:
                    # print(f"Katılımcı {p_idx+1}, Deneme {t_idx+1} ECG hücre içeriği beklenmedik tip: {type(ecg_cell_content)}")
                    total_trials_skipped_access_error += 1
                    continue # Bu denemeyi atla

            except Exception as e_cell_access:
                # print(f"Katılımcı {p_idx+1}, Deneme {t_idx+1} hücre içeriği alınırken hata ({e_cell_access}). Atlanıyor.")
                total_trials_skipped_access_error += 1
                continue

            if eeg_stimuli_raw is None or ecg_stimuli_raw is None or \
               eeg_stimuli_raw.shape[0] == 0 or ecg_stimuli_raw.shape[0] == 0:
                # print(f"Katılımcı {p_idx+1}, Deneme {t_idx+1} ham sinyal boş veya alınamadı. Atlanıyor.")
                total_trials_skipped_access_error += 1
                continue

            if eeg_stimuli_raw.shape[0] < MIN_SIGNAL_LENGTH_FOR_FILTER or \
               ecg_stimuli_raw.shape[0] < (MIN_SIGNAL_LENGTH_FOR_FILTER * ECG_FS_ORIGINAL / TARGET_FS):
                total_trials_skipped_short_signal += 1
                continue

            eeg_stimuli_filtered = butter_bandpass_filter(eeg_stimuli_raw, LOWCUT, HIGHCUT, EEG_FS)

            num_samples_original_ecg = ecg_stimuli_raw.shape[0]
            num_samples_target_ecg = int(num_samples_original_ecg * TARGET_FS / ECG_FS_ORIGINAL)
            if num_samples_target_ecg < MIN_SIGNAL_LENGTH_FOR_FILTER:
                total_trials_skipped_short_signal += 1
                continue

            # EKG kanallarının (genellikle 2) doğru olduğundan emin ol
            if ecg_stimuli_raw.shape[1] == 0:
                # print(f"Katılımcı {p_idx+1}, Deneme {t_idx+1} EKG sinyalinde kanal yok. Atlanıyor.")
                total_trials_skipped_access_error += 1
                continue

            ecg_downsampled = np.zeros((num_samples_target_ecg, ecg_stimuli_raw.shape[1]))
            for ch_idx in range(ecg_stimuli_raw.shape[1]):
                 ecg_downsampled[:, ch_idx] = resample(ecg_stimuli_raw[:, ch_idx], num_samples_target_ecg)
            ecg_stimuli_filtered = butter_bandpass_filter(ecg_downsampled, LOWCUT, HIGHCUT, TARGET_FS)

            if (eeg_stimuli_filtered is eeg_stimuli_raw and eeg_stimuli_raw.shape[0] < MIN_SIGNAL_LENGTH_FOR_FILTER) or \
               (ecg_stimuli_filtered is ecg_downsampled and ecg_downsampled.shape[0] < MIN_SIGNAL_LENGTH_FOR_FILTER):
                total_trials_skipped_short_signal += 1
                continue

            min_trial_segment_len = min(eeg_stimuli_filtered.shape[0], ecg_stimuli_filtered.shape[0])
            if min_trial_segment_len == 0:
                total_trials_skipped_short_signal +=1
                continue

            eeg_stimuli_aligned = eeg_stimuli_filtered[:min_trial_segment_len, :]
            ecg_stimuli_aligned = ecg_stimuli_filtered[:min_trial_segment_len, :]

            if min_trial_segment_len > max_len_overall: max_len_overall = min_trial_segment_len

            combined_features_for_trial = np.hstack((eeg_stimuli_aligned, ecg_stimuli_aligned))
            participant_trials_data_list.append(combined_features_for_trial)

            if t_idx < len(valence_scores_for_participant) and t_idx < len(arousal_scores_for_participant):
                participant_trials_valence_labels.append(1 if valence_scores_for_participant[t_idx] >= LABEL_THRESHOLD else 0)
                participant_trials_arousal_labels.append(1 if arousal_scores_for_participant[t_idx] >= LABEL_THRESHOLD else 0)
                total_trials_processed_successfully +=1
            else:
                # print(f"Katılımcı {p_idx+1}, Deneme {t_idx+1} için skor bulunamadı (indeks dışı). Atlanıyor.")
                if participant_trials_data_list: participant_trials_data_list.pop() # Eklenen veriyi geri al
                total_trials_skipped_access_error +=1

        if not participant_trials_data_list:
            all_participants_data_list.append([])
            all_participants_valence_labels_list.append(np.array([]))
            all_participants_arousal_labels_list.append(np.array([]))
        else:
            all_participants_data_list.append(participant_trials_data_list)
            all_participants_valence_labels_list.append(np.array(participant_trials_valence_labels))
            all_participants_arousal_labels_list.append(np.array(participant_trials_arousal_labels))

    print(f"\nVeri yükleme ve ön işleme tamamlandı.")
    print(f"Toplam {total_trials_processed_successfully} deneme başarıyla işlendi.")
    print(f"Toplam {total_trials_skipped_short_signal} deneme sinyal kısalığı nedeniyle atlandı.")
    print(f"Toplam {total_trials_skipped_access_error} deneme veri erişim/yapı hatası nedeniyle atlandı.")
    print(f"Bulunan maksimum dizi uzunluğu (padding için): {max_len_overall}")

    if max_len_overall == 0 and total_trials_processed_successfully == 0 :
        print("KRİTİK HATA: Hiçbir katılımcıdan geçerli veri işlenemedi. max_len_overall sıfır.")
        return None, None, None, 0
    return all_participants_data_list, all_participants_valence_labels_list, all_participants_arousal_labels_list, max_len_overall

# ----- create_cnn_lstm_model ve Ana Kod Akışı (öncekiyle aynı) -----
def create_cnn_lstm_model(input_shape_tuple):
    model = Sequential(name="CNN_LSTM_Emotion_Classifier")
    model.add(Input(shape=input_shape_tuple))
    model.add(Conv1D(filters=64, kernel_size=5, activation='relu', name='conv1_1'))
    model.add(MaxPooling1D(pool_size=3, name='pool1_1'))
    model.add(Dropout(0.2, name='dropout1_1'))
    model.add(Conv1D(filters=48, kernel_size=5, activation='relu', name='conv1_2'))
    model.add(MaxPooling1D(pool_size=3, name='pool1_2'))
    model.add(Dropout(0.2, name='dropout1_2'))
    model.add(Conv1D(filters=24, kernel_size=3, activation='relu', name='conv1_3'))
    model.add(MaxPooling1D(pool_size=3, name='pool1_3'))
    model.add(Dropout(0.2, name='dropout1_3'))
    model.add(LSTM(128, return_sequences=True, name='lstm_1'))
    model.add(LSTM(64, name='lstm_2'))
    model.add(Dense(1, activation='sigmoid', name='output_dense'))
    try:
        optimizer = RMSprop(learning_rate=LR)
    except TypeError: # Eski TF sürümleri için
        optimizer = tf.keras.optimizers.legacy.RMSprop(learning_rate=LR)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model

# ----- Ana Kod Akışı -----
if not os.path.exists(DREAMER_MAT_PATH_DRIVE):
    print(f"HATA: DREAMER.mat dosyası {DREAMER_MAT_PATH_DRIVE} konumunda bulunamadı.")
else:
    print("DREAMER veri seti yükleniyor ve ön işleniyor...")
    all_participants_processed_data, all_participants_valence_labels, \
    all_participants_arousal_labels, max_sequence_len = load_and_preprocess_dreamer(DREAMER_MAT_PATH_DRIVE)

    if all_participants_processed_data is None or max_sequence_len == 0:
        print("Veri yükleme başarısız oldu veya işlenecek veri bulunamadı. Program sonlandırılıyor.")
    else:
        num_total_features = 0
        first_valid_participant_for_features = -1
        # Özellik sayısını ve ilk geçerli katılımcıyı bul
        for i, p_data_list in enumerate(all_participants_processed_data):
            if p_data_list: # Katılımcının deneme listesi boş değilse
                # Bu listedeki ilk geçerli (boş olmayan) denemeyi bul
                for trial_data in p_data_list:
                    if isinstance(trial_data, np.ndarray) and trial_data.shape[0] > 0 and trial_data.ndim == 2:
                        num_total_features = trial_data.shape[1]
                        first_valid_participant_for_features = i
                        break # İç döngüden çık
                if num_total_features > 0:
                    break # Dış döngüden çık

        if num_total_features == 0:
             print("UYARI: Özellik sayısı belirlenemedi (num_total_features = 0). Model eğitimi atlanıyor.")
        else:
            print(f"Özellik sayısı: {num_total_features}, Padding için maksimum dizi uzunluğu: {max_sequence_len}")
            participant_specific_valence_accuracies = []
            participant_specific_arousal_accuracies = []
            valid_participants_count_for_training = 0 # Model eğitimi yapılan katılımcı sayısı
            print("\n--- Deney 1 Başlıyor: Her katılımcı için ayrı eğitim ve test ---")

            for p_idx in range(N_PARTICIPANTS):
                current_participant_data_trials = all_participants_processed_data[p_idx]
                current_participant_valence_labels = all_participants_valence_labels[p_idx] # Bu bir numpy array olmalı
                current_participant_arousal_labels = all_participants_arousal_labels[p_idx] # Bu da

                # Etiketlerin de numpy array olduğundan ve boş olmadığından emin ol
                if not isinstance(current_participant_valence_labels, np.ndarray) or \
                   not isinstance(current_participant_arousal_labels, np.ndarray) or \
                   current_participant_valence_labels.size == 0 or \
                   current_participant_arousal_labels.size == 0:
                    # print(f"Katılımcı {p_idx + 1} için etiketler eksik veya geçersiz. Atlanıyor.")
                    continue

                # Veri ve etiket sayısının eşleştiğinden emin ol
                if not current_participant_data_trials or \
                   len(current_participant_data_trials) != len(current_participant_valence_labels) or \
                   len(current_participant_data_trials) < (TRAIN_TRIALS_MIN_REQUIRED + TEST_TRIALS_MIN_REQUIRED):
                    # print(f"Katılımcı {p_idx + 1} için yeterli/uyumlu veri/etiket yok ({len(current_participant_data_trials)} veri, {len(current_participant_valence_labels)} etiket). Atlanıyor.")
                    continue

                trial_indices_for_participant = np.arange(len(current_participant_data_trials))
                n_trials_for_participant = len(current_participant_data_trials)

                current_test_size = TEST_TRIALS_MIN_REQUIRED
                # Test boyutunu, mevcut deneme sayısına ve beklenen orana göre ayarla
                potential_test_size = int(round(n_trials_for_participant * (4.0/N_TRIALS_PER_PARTICIPANT_EXPECTED)))
                if potential_test_size >= TEST_TRIALS_MIN_REQUIRED:
                    current_test_size = potential_test_size

                # Eğitim için yeterli deneme kalıp kalmadığını kontrol et
                if n_trials_for_participant - current_test_size < TRAIN_TRIALS_MIN_REQUIRED:
                    # print(f"Katılımcı {p_idx + 1} için test boyutu ({current_test_size}) sonrası yeterli eğitim denemesi ({n_trials_for_participant - current_test_size}) kalmıyor. Atlanıyor.")
                    continue

                try:
                    # Etiketlerin en az iki farklı sınıfa sahip olup olmadığını kontrol et (stratify için)
                    stratify_val = current_participant_valence_labels if len(np.unique(current_participant_valence_labels)) > 1 else None
                    train_trial_indices_v, test_trial_indices_v = train_test_split(
                        trial_indices_for_participant, test_size=current_test_size, random_state=SEED, shuffle=True, stratify=stratify_val)

                    stratify_aro = current_participant_arousal_labels if len(np.unique(current_participant_arousal_labels)) > 1 else None
                    train_trial_indices_a, test_trial_indices_a = train_test_split(
                        trial_indices_for_participant, test_size=current_test_size, random_state=SEED, shuffle=True, stratify=stratify_aro)

                except ValueError as e_split: # Stratify başarısız olursa
                    # print(f"Katılımcı {p_idx + 1} için stratify ile bölme hatası ({e_split}). Basit bölme deneniyor.")
                    try:
                        train_trial_indices_v, test_trial_indices_v = train_test_split(
                            trial_indices_for_participant, test_size=current_test_size, random_state=SEED, shuffle=True)
                        train_trial_indices_a, test_trial_indices_a = train_test_split(
                            trial_indices_for_participant, test_size=current_test_size, random_state=SEED, shuffle=True)
                    except Exception as e_split_again:
                        # print(f"Katılımcı {p_idx + 1} için ikinci train_test_split denemesinde de hata ({e_split_again}). Atlanıyor.")
                        continue

                # Valence için
                X_train_v_list = [current_participant_data_trials[i] for i in train_trial_indices_v]
                X_test_v_list = [current_participant_data_trials[i] for i in test_trial_indices_v]
                y_train_v_labels = current_participant_valence_labels[train_trial_indices_v]
                y_test_v_labels = current_participant_valence_labels[test_trial_indices_v]

                # Arousal için
                X_train_a_list = [current_participant_data_trials[i] for i in train_trial_indices_a]
                X_test_a_list = [current_participant_data_trials[i] for i in test_trial_indices_a]
                y_train_a_labels = current_participant_arousal_labels[train_trial_indices_a]
                y_test_a_labels = current_participant_arousal_labels[test_trial_indices_a]


                if not X_train_v_list or not X_test_v_list or not X_train_a_list or not X_test_a_list :
                    # print(f"Katılımcı {p_idx + 1} için eğitim veya test listeleri boş. Atlanıyor.")
                    continue

                # Valence için ölçekleme ve padding
                temp_train_v_concat = np.vstack(X_train_v_list)
                if temp_train_v_concat.shape[0] == 0: continue
                scaler_v = MinMaxScaler().fit(temp_train_v_concat)
                X_train_v_scaled = [scaler_v.transform(trial) for trial in X_train_v_list]
                X_test_v_scaled = [scaler_v.transform(trial) for trial in X_test_v_list]
                X_train_v_padded = pad_sequences(X_train_v_scaled, maxlen=max_sequence_len, dtype='float32', padding='post', truncating='post')
                X_test_v_padded = pad_sequences(X_test_v_scaled, maxlen=max_sequence_len, dtype='float32', padding='post', truncating='post')

                # Arousal için ölçekleme ve padding
                temp_train_a_concat = np.vstack(X_train_a_list)
                if temp_train_a_concat.shape[0] == 0: continue
                scaler_a = MinMaxScaler().fit(temp_train_a_concat)
                X_train_a_scaled = [scaler_a.transform(trial) for trial in X_train_a_list]
                X_test_a_scaled = [scaler_a.transform(trial) for trial in X_test_a_list]
                X_train_a_padded = pad_sequences(X_train_a_scaled, maxlen=max_sequence_len, dtype='float32', padding='post', truncating='post')
                X_test_a_padded = pad_sequences(X_test_a_scaled, maxlen=max_sequence_len, dtype='float32', padding='post', truncating='post')


                if X_train_v_padded.shape[0] == 0 or X_test_v_padded.shape[0] == 0 or \
                   X_train_a_padded.shape[0] == 0 or X_test_a_padded.shape[0] == 0:
                    # print(f"Katılımcı {p_idx + 1} için padding sonrası veri boş. Atlanıyor.")
                    continue

                print(f"Katılımcı {p_idx + 1}: V(Eğt {X_train_v_padded.shape[0]}, Tst {X_test_v_padded.shape[0]}), A(Eğt {X_train_a_padded.shape[0]}, Tst {X_test_a_padded.shape[0]})")
                valid_participants_count_for_training += 1
                model_input_shape = (max_sequence_len, num_total_features)

                # Valence Modeli
                model_valence = create_cnn_lstm_model(model_input_shape)
                if p_idx == first_valid_participant_for_features : model_valence.summary()
                history_valence = model_valence.fit(X_train_v_padded, y_train_v_labels, epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=0, validation_data=(X_test_v_padded, y_test_v_labels))
                loss_v, acc_v = model_valence.evaluate(X_test_v_padded, y_test_v_labels, verbose=0)
                participant_specific_valence_accuracies.append(acc_v)

                # Arousal Modeli
                model_arousal = create_cnn_lstm_model(model_input_shape)
                history_arousal = model_arousal.fit(X_train_a_padded, y_train_a_labels, epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=0, validation_data=(X_test_a_padded, y_test_a_labels))
                loss_a, acc_a = model_arousal.evaluate(X_test_a_padded, y_test_a_labels, verbose=0)
                participant_specific_arousal_accuracies.append(acc_a)

                tf.keras.backend.clear_session() # Belleği temizle

            if not participant_specific_valence_accuracies: # Eğer hiç geçerli katılımcı işlenemediyse
                avg_valence_accuracy_exp1, std_valence_accuracy_exp1 = 0.0, 0.0
                avg_arousal_accuracy_exp1, std_arousal_accuracy_exp1 = 0.0, 0.0
            else:
                avg_valence_accuracy_exp1 = np.mean(participant_specific_valence_accuracies)
                std_valence_accuracy_exp1 = np.std(participant_specific_valence_accuracies)
                avg_arousal_accuracy_exp1 = np.mean(participant_specific_arousal_accuracies)
                std_arousal_accuracy_exp1 = np.std(participant_specific_arousal_accuracies)

            print("\n\n--- Deney 1 Özet Sonuçları ---")
            print(f"Toplam {valid_participants_count_for_training} katılımcı için model eğitildi.")
            print(f"Ortalama Valence Doğruluğu: {avg_valence_accuracy_exp1*100:.2f}% (Std: {std_valence_accuracy_exp1*100:.2f}%)")
            print(f"Ortalama Arousal Doğruluğu: {avg_arousal_accuracy_exp1*100:.2f}% (Std: {std_arousal_accuracy_exp1*100:.2f}%)")

Mounted at /content/drive
DREAMER veri seti yükleniyor ve ön işleniyor...
/content/drive/MyDrive/DREAMER.mat yükleniyor...
Katılımcı 1/23 işleniyor...

--- Katılımcı 1 Veri Yapısı İncelemesi (Düzeltme Sonrası) ---
EEG L1 container tipi: <class 'numpy.ndarray'>, şekli: (1, 1)
EEG L2 container (iç hücre dizisi) tipi: <class 'numpy.ndarray'>, şekli: (18, 1)
EEG L2'nin ilk hücresinin (gerçek sinyal hücresi) tipi: <class 'numpy.ndarray'>
EEG gerçek sinyal (ilk deneme) tipi: <class 'numpy.ndarray'>, şekli: (25472, 14)
--- Bitti: Veri Yapısı İncelemesi (Düzeltme Sonrası) ---

Katılımcı 2/23 işleniyor...
Katılımcı 3/23 işleniyor...
Katılımcı 4/23 işleniyor...
Katılımcı 5/23 işleniyor...
Katılımcı 6/23 işleniyor...
Katılımcı 7/23 işleniyor...
Katılımcı 8/23 işleniyor...
Katılımcı 9/23 işleniyor...
Katılımcı 10/23 işleniyor...
Katılımcı 11/23 işleniyor...
Katılımcı 12/23 işleniyor...
Katılımcı 13/23 işleniyor...
Katılımcı 14/23 işleniyor...
Katılımcı 15/23 işleniyor...
Katılımcı 16/23 işleniyor.

Model: "CNN_LSTM_Emotion_Classifier"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1_1 (Conv1D)                │ (None, 50428, 64)      │         5,184 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ pool1_1 (MaxPooling1D)          │ (None, 16809, 64)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout1_1 (Dropout)            │ (None, 16809, 64)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1_2 (Conv1D)                │ (None, 16805, 48)      │        15,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ pool1_2 (MaxPooling1D)          │ (None, 5601, 48)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout1_2 (Dropout)            │ (None, 5601, 48)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1_3 (Conv1D)                │ (None, 5599, 24)       │         3,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ pool1_3 (MaxPooling1D)          │ (None, 1866, 24)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout1_3 (Dropout)            │ (None, 1866, 24)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 1866, 128)      │        78,336 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 64)             │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output_dense (Dense)            │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 151,881 (593.29 KB)

 Trainable params: 151,881 (593.29 KB)

 Non-trainable params: 0 (0.00 B)

Katılımcı 2: V(Eğt 14, Tst 4), A(Eğt 14, Tst 4)


Katılımcı 3: V(Eğt 14, Tst 4), A(Eğt 14, Tst 4)
Katılımcı 4: V(Eğt 14, Tst 4), A(Eğt 14, Tst 4)
Katılımcı 5: V(Eğt 14, Tst 4), A(Eğt 14, Tst 4)
Katılımcı 6: V(Eğt 14, Tst 4), A(Eğt 14, Tst 4)
Katılımcı 7: V(Eğt 14, Tst 4), A(Eğt 14, Tst 4)
Katılımcı 8: V(Eğt 14, Tst 4), A(Eğt 14, Tst 4)
Katılımcı 9: V(Eğt 14, Tst 4), A(Eğt 14, Tst 4)
Katılımcı 10: V(Eğt 14, Tst 4), A(Eğt 14, Tst 4)
Katılımcı 11: V(Eğt 14, Tst 4), A(Eğt 14, Tst 4)
Katılımcı 12: V(Eğt 14, Tst 4), A(Eğt 14, Tst 4)
Katılımcı 13: V(Eğt 14, Tst 4), A(Eğt 14, Tst 4)
Katılımcı 14: V(Eğt 14, Tst 4), A(Eğt 14, Tst 4)
Katılımcı 15: V(Eğt 14, Tst 4), A(Eğt 14, Tst 4)
Katılımcı 16: V(Eğt 14, Tst 4), A(Eğt 14, Tst 4)
Katılımcı 17: V(Eğt 14, Tst 4), A(Eğt 14, Tst 4)
Katılımcı 18: V(Eğt 14, Tst 4), A(Eğt 14, Tst 4)
Katılımcı 19: V(Eğt 14, Tst 4), A(Eğt 14, Tst 4)
Katılımcı 20: V(Eğt 14, Tst 4), A(Eğt 14, Tst 4)
Katılımcı 21: V(Eğt 14, Tst 4), A(Eğt 14, Tst 4)
Katılımcı 22: V(Eğt 14, Tst 4), A(Eğt 14, Tst 4)
Katılımcı 23: V(Eğt 14, Tst